In [1]:
from lusidtools.jupyter_tools import toggle_code

# Calculating Intraday P&L for CFD's with and without Daily Close outs

In this notebook, we demonstrate how a Contract for Difference (CFD) can be booked in LUSID.
We will also demonstrate how you can then value this instrument and calculate it's PnL in both the non-close out and close out method.

## AMZN CFD with Unrealized P&L ('non-close out')
For our non-close out CFD we will create a CFD on Amazon stock. This will just have a regular valuation and the PnL will be unrealized.

## AAPL CFD with Realized P&L ('close out)
For our close out CFD we will create a CFD on Apple stock. This will "realize" the PnL every day, and book it as a cash position while raising the cost basis of the Apple CFD to reflect the action of taking out the realized PnL.

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.8157.0


In [3]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)

In [4]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"
portfolio_code = "CFDPortfolioForPnLCalc"

# 1. Create Portfolio

Create the portfolio that will contain the CFDs.

In [5]:
portfolio_code = "CFDPortfolioForPnLCalc"

try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id CFDPortfolioForPnLCalc because it already exists in scope ibor.


# 2. Create Instruments

## 2.1 Create the underlying instruments

In order to create a derivative on an instrument, we must first create that underlying instrument. As we are planning to create CFDs based on Amazon and Apple, we will upsert the underlying stocks first.

In [21]:
# upload AMZN and AAPL Equities

equity = lm.SimpleInstrument(
    instrument_type="SimpleInstrument", 
    dom_ccy="USD", 
    asset_class="Equities", 
    simple_instrument_type="Equity"
)

equity_definition = lm.InstrumentDefinition(
    name="Amazon.com",
    identifiers={"ClientInternal": lm.InstrumentIdValue("AMZN"), "RIC": lm.InstrumentIdValue("AMZN")},
    definition=equity,
)

# upsert the instrument
upsert_request = {"AMZN": equity_definition}
upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
equity_luid = upsert_response.values["AMZN"].lusid_instrument_id
print(equity_luid)

equity = lm.SimpleInstrument(
    instrument_type="SimpleInstrument", 
    dom_ccy="USD", 
    asset_class="Equities", 
    simple_instrument_type="Equity"
)

equity_definition = lm.InstrumentDefinition(
    name="Apple",
    identifiers={"ClientInternal": lm.InstrumentIdValue("AAPL")},
    definition=equity,
)

# upsert the instrument
upsert_request = {"AAPL": equity_definition}
upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
equity_luid = upsert_response.values["AAPL"].lusid_instrument_id
print(equity_luid)

LUID_0000CZRI
LUID_0000D662


## 2.2 Create the CFDs

Now that we have created the underlying stocks, we can create a function to upsert CFDs in LUSID.

In [7]:
def create_cfd(
    cfd_name, 
    cfd_identifier, 
    start_date,
    contract_size, 
    pay_ccy, 
    underlying_ccy, 
    underlying_code,
    underlying_identifier,
    reference_rate, 
):

    cfd = lm.ContractForDifference( 
        start_date=start_date,
        contract_size=contract_size,
        pay_ccy=pay_ccy,
        underlying_ccy=underlying_ccy,
        code=underlying_code,
        underlying_identifier=underlying_identifier,
        reference_rate=reference_rate,
        type="Cash",
        instrument_type="ContractForDifference",
    )

    # define the instrument to be upserted
    cfd_definition = lm.InstrumentDefinition(
        name=cfd_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(cfd_identifier)},
        definition=cfd,
    )

    # upsert the instrument
    upsert_request = {cfd_identifier: cfd_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    cfd_luid = upsert_response.values[cfd_identifier].lusid_instrument_id
    print(cfd_luid)

After creating the function that will upsert CFDs, we can now use this function to create an Amazon and Apple CFD.

In [8]:
    cfd_name = "AMZN CFD"
    cfd_identifier = "AMZN_CFD"
    start_date = datetime(2021, 9, 1, 00, tzinfo=pytz.utc)
    contract_size = 50
    pay_ccy = "USD"
    underlying_ccy = "USD"
    underlying_code = "LUID_0000CZRI"
    underlying_identifier = "LusidInstrumentId"
    reference_rate = 3479

    create_cfd(
        cfd_name, 
        cfd_identifier, 
        start_date,
        contract_size, 
        pay_ccy, 
        underlying_ccy, 
        underlying_code,
        underlying_identifier, 
        reference_rate
    )


LUID_0000CZRJ


In [9]:
    cfd_name = "AAPL CFD"
    cfd_identifier = "AAPL_CFD"
    start_date = datetime(2021, 9, 1, 00, tzinfo=pytz.utc)
    contract_size = 100
    pay_ccy = "USD"
    underlying_ccy = "USD"
    underlying_code = "LUID_0000D662"
    underlying_identifier = "LusidInstrumentId"
    reference_rate = 152

    create_cfd(
        cfd_name, 
        cfd_identifier, 
        start_date,
        contract_size, 
        pay_ccy, 
        underlying_ccy, 
        underlying_code,
        underlying_identifier, 
        reference_rate
    )

LUID_0000D663


# 3. Transactions

We have to generate transaction types to reflect the opening of a new CFD contract and to realise a change in PnL for the close out CFD.

LUSID allows us to use custom movements, we structure the OpenContract type which just has to add the contract to our portfolio. We also create the RealisePnLIncrease type, which will increase the cost basis of our position and increase the cash by the realized PnL of that day.

## 3.1 Create Transaction Types

In [10]:
new_transaction_config = [
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="OpenContract",
                description="A CFD transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    ),
    lm.TransactionConfigurationDataRequest(    
        aliases=[        
            lm.TransactionConfigurationTypeAlias(
                type="RealisePnLIncrease",
                description="Increase cost basis and adjust cash",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
            ),lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    ),
    lm.TransactionConfigurationDataRequest(    
        aliases=[        
            lm.TransactionConfigurationTypeAlias(
                type="RealisePnLDecrease",
                description="Decrease cost basis and adjust cash",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=-1,
                properties=None,
                mappings=[],
            ),lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

## 3.2 Book Transactions

We book in our transactions, note that we close out our Apple position on a daily basis, so once acquired we use our RealisePnLIncrease transaction type each morning to Realize the PnL of the previous day.

In [11]:
transactions = pd.read_csv("data/cfd_transaction_data.csv")
transactions

,txn_id,type,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency,portfolio
0,txn001,OpenContract,AMZN_CFD,2021-09-01T08:00:00Z,2021-09-01T08:00:00Z,100,"3,479.00","350,000.00",USD,CFDPortfolioForPnLCalc
1,co_txn001,OpenContract,AAPL_CFD,2021-09-01T08:00:00Z,2021-09-01T08:00:00Z,100,152,"15,200.00",USD,CFDPortfolioForPnLCalc
2,co_rpnlb_txn001,RealisePnLIncrease,AAPL_CFD,2021-09-30T08:00:00Z,2021-09-30T08:00:00Z,0,0,"-120,000.00",USD,CFDPortfolioForPnLCalc
3,co_rpnlb_txn002,RealisePnLIncrease,AAPL_CFD,2021-10-30T08:00:00Z,2021-10-30T08:00:00Z,0,0,"120,000.00",USD,CFDPortfolioForPnLCalc


In [12]:
for portfolio_code, grouped_df in transactions.groupby('portfolio'):

    transaction_request=[
            lm.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["type"],
                instrument_identifiers={
                    "Instrument/default/ClientInternal" :row["client_id"]
                },
                transaction_date=row["trade_date"],
                settlement_date=row["settlement_date"],
                units=row["quantity"],
                transaction_price=lm.TransactionPrice(
                    price=row["price"], type="Price"
                ),
                total_consideration=lm.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency"]
                ),
            ) for index, row in grouped_df.iterrows()
        ]

    response = transaction_portfolios_api.upsert_transactions(
        scope=scope,
        code=portfolio_code,
        transaction_request=transaction_request
      )

    print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2021-11-01 15:43:26.530448+00:00


# 4. Quotes

For our CFD to work, we will need quotes for the underlying instruments. Here, we will upload quotes for Amazon and Apple 3 times a day (at 8:00, 12:00 and 17:00). We use the Luid's we generated when creating the instrument in part 2.1 of this notebook.

In [13]:
underlying_prices = pd.read_csv("data/cfd_underlying_quotes_data.csv")
underlying_prices.head()

,luid,date,price,currency
0,LUID_0000CZRI,2021-09-01T08:00:00Z,"3,479.00",USD
1,LUID_0000CZRI,2021-09-02T08:00:00Z,"3,480.00",USD
2,LUID_0000CZRI,2021-09-30T08:00:00Z,"3,250.00",USD
3,LUID_0000CZRI,2021-10-30T08:00:00Z,"3,450.00",USD
4,LUID_0000D662,2021-09-01T08:00:00Z,152,USD


In [14]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["luid"],
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="mid",
            ),
            effective_at=to_date(row["date"]),
        ),
        metric_value=lm.MetricValue(value=row["price"], unit=row["currency"]),
        #scale_factor=100,
    )
    for index, row in underlying_prices.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(
        f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

Quote successfully loaded into LUSID. 8 quotes loaded.


# 5. Valuations

Once we have created the CFDs and loaded them in our portfolio, we can now value the portfolio. The recipe below describes how we will go about valuing these instruments. It details which model we are using, in this case the "Constant Time Value of Money" model supplied by Lusid.

## 5.1 Create Valuation Recipe

In [15]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="cfdValuation",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Equity.*.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        suppliers=lm.MarketContextSuppliers(
            commodity="Lusid", credit="Lusid", equity="Lusid", fx="Lusid", rates="Lusid"
        ),
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="ContractForDifference",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 5.2 Create Valuation Function

Once we have made a recipe, we can now create a function that outputs a dataframe with the valuation of our portfolio.

In [16]:
def get_daily_cfd_val(date, portfolio_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code="cfdValuation"),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            #lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Valuation/Exposure/Amount", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value"),
            lm.AggregateSpec("Holding/default/Cost", "Value")
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            #"Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Present Value",
            "Valuation/Exposure/Amount": "Exposure",
            "Valuation/PnL/Tm1": "PnL (1-day)",
        },
        inplace=True,
    )

    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

## 5.3 Valuations

### Day 1



In [17]:
get_daily_cfd_val("2021-09-01T08:00:00Z", portfolio_code)

,Present Value,Exposure,InstrumentName,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day),Holding/default/Cost
0,0.00,0.00,AMZN CFD,50.00,"3,479.00",100.00,None,"350,000.00"
1,0.00,0.00,AAPL CFD,100.00,152.00,100.00,None,"15,200.00"


### Day 2

In [18]:
get_daily_cfd_val("2021-09-02T08:00:00Z", portfolio_code)

,Present Value,Exposure,InstrumentName,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day),Holding/default/Cost
0,"5,000.00","5,000.00",AMZN CFD,50.00,"3,480.00",100.00,"5,000.00","350,000.00"
1,"10,000.00","10,000.00",AAPL CFD,100.00,153.00,100.00,"10,000.00","15,200.00"


### A month later

In [19]:
get_daily_cfd_val("2021-09-30T08:00:00Z", portfolio_code)


,Present Value,Exposure,InstrumentName,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day),Holding/default/Cost
0,"-1,717,500.00","-1,717,500.00",AMZN CFD,50.00,"3,250.00",150.00,nan,"523,000.00"
1,"-180,000.00","-180,000.00",AAPL CFD,100.00,140.00,150.00,"-42,450.00","-97,250.00"
2,"-120,000.00","-120,000.00",USD,1.00,nan,"-120,000.00",0.00,"-120,000.00"


### Two months later

In [20]:
get_daily_cfd_val("2021-10-30T08:00:00Z", portfolio_code)

,Present Value,Exposure,InstrumentName,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day),Holding/default/Cost
0,"-217,500.00","-217,500.00",AMZN CFD,50.00,"3,450.00",150.00,nan,"523,000.00"
1,0.00,0.00,AAPL CFD,100.00,152.00,150.00,"33,000.00","22,750.00"
